In [ ]:
import sys
!{sys.executable} -m pip install rdflib

In [1]:
from rdflib import Graph
from bs4 import *
import requests
import os
from os import walk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import sys
from rdflib import URIRef,Literal
from urllib.request import urlopen
import uuid
import multiprocessing as mp
import gc
import random

### Data Loading

In [2]:
'''Loads in all the comics. paths of the files 
in the comic folders that dont end in either: 
.jpg, .png or jpeg are not saved.'''

path = r"/data/decompressed"

files = []
comics = [x[1] for x in os.walk(path)][0]
for (root, dirnames, filenames) in walk(path):
    for fn in filenames:
        if not fn.lower().endswith((".jpg", ".png", ".jpeg")):
            continue
        else:
            files.append(os.path.join(root,fn))

In [9]:
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
'''Convert all page paths to Image objects and store them'''
def get_images(paths, url, directory):
    create_directory(directory)
    ex = 0
    if paths != []:
        if url:
            for p in paths:
                try:
                    im = Image.open(requests.get(p, stream=True).raw).convert('RGB')
                except:
                    ex += 1
                    print(ex)
                    continue
                    
                name = uuid.uuid4().hex
                im.save(directory+name+".jpg")
                im.close
        else:
            for p in paths:
                im = Image.open(p).convert('RGB')

                name = uuid.uuid4().hex
                im.save(directory+name+".jpg")
                im.close

In [3]:
%%time
g = Graph()
data = "https://data.spinque.com/ld/data/netwerkoorlogsbronnen/beeldbank_niod/data/export.nt"
g.parse(data)

CPU times: user 3min 33s, sys: 11.4 s, total: 3min 44s
Wall time: 4min 28s


<Graph identifier=Nbebe9eab88e848479ca6a2dcba8a6d1f (<class 'rdflib.graph.Graph'>)>

In [4]:
%%time
thumbnails = [u.n3()[1:-1] for _,_,u in g if "https://images.memorix.nl/niod/thumb/" in u]

CPU times: user 6.69 s, sys: 1.56 s, total: 8.25 s
Wall time: 8.68 s


In [ ]:
%%time
get_images(files, False, "/data/comics/")

In [5]:
thumbnails_sample = random.sample(thumbnails, 1000)

In [10]:
%%time
get_images(thumbnails_sample, True, "/data/photos_testset_noise/")

CPU times: user 53.6 s, sys: 4.38 s, total: 58 s
Wall time: 3min 49s


### LEGACY